<a href="https://colab.research.google.com/github/Cesc-Fran/meteo/blob/main/tests/test_visualcrossing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Test visual crossing

https://www.visualcrossing.com/resources/documentation/weather-api/timeline-weather-api/


revision :
- author : Fran
- date.  : 30 july 2025
- rev    : 10 - first test  



In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
import time

from google.colab import userdata

In [2]:
# Securely retrieve API key (e.g., from environment variables)
# For Visual Crossing, replace 'YOUR_VISUAL_CROSSING_API_KEY' with your actual key
# It is highly recommended to set this as an environment variable, e.g., VC_API_KEY

# VISUAL_CROSSING_API_KEY = os.getenv('VC_API_KEY', 'YOUR_VISUAL_CROSSING_API_KEY_HERE')

VISUAL_CROSSING_API_KEY = userdata.get('VC_API_KEY')

In [3]:
# Function to get historical weather data

def fetch_weather_data_vc(city, start_date, end_date, api_key):
    """
    Fetches daily historical weather data from Visual Crossing API.
    Args:
        city (str): Name of the city (e.g., "Madrid, Spain").
        start_date (str): Start date in YYYY-MM-DD format.
        end_date (str): End date in YYYY-MM-DD format.
        api_key (str): Your Visual Crossing API key.
    Returns:
        dict: JSON response from the API, or None if an error occurs.
    """
    base_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline"
    params = {
        "unitGroup": "metric",
        "contentType": "json",
        "include": "days", # Request daily summaries as specified in documentation [1]
        "key": api_key
    }
    url = f"{base_url}/{city}/{start_date}/{end_date}"

    try:
        response = requests.get(url, params=params)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred for {city} ({start_date} to {end_date}): {http_err}")
        if response.status_code == 429:
            print("Rate limit exceeded. Consider pausing or increasing your plan.")
        return None
    except requests.exceptions.ConnectionError as conn_err:
        print(f"Connection error occurred for {city} ({start_date} to {end_date}): {conn_err}")
        return None
    except requests.exceptions.Timeout as timeout_err:
        print(f"Timeout error occurred for {city} ({start_date} to {end_date}): {timeout_err}")
        return None
    except requests.exceptions.RequestException as req_err:
        print(f"An unexpected error occurred for {city} ({start_date} to {end_date}): {req_err}")
        return None

# Example usage (will be integrated into the full script below)
# madrid_data = fetch_weather_data_vc("Madrid, Spain", "2023-09-01", "2023-10-31", VISUAL_CROSSING_API_KEY)
# if madrid_data:
#     print(f"Successfully fetched data for Madrid. Days: {len(madrid_data.get('days',))}")

In [5]:
# test with Montpellier as city

mpl_data = fetch_weather_data_vc("Montpellier, France", "2025-06-15", "2025-07-31", VISUAL_CROSSING_API_KEY)
if mpl_data:
   print(f"Successfully fetched data for Montpellier. Days: {len(mpl_data.get('days',))}")

Successfully fetched data for Montpellier. Days: 47


In [29]:
mpl_data.keys()

dict_keys(['queryCost', 'latitude', 'longitude', 'resolvedAddress', 'address', 'timezone', 'tzoffset', 'days', 'stations'])

In [28]:
mpl_data['stations']

{'30003001': {'distance': 27918.0,
  'latitude': 43.537,
  'longitude': 4.207,
  'useCount': 0,
  'id': '30003001',
  'name': 'AIGUES-MORTES',
  'quality': 100,
  'contribution': 0.0},
 '34154001': {'distance': 8117.0,
  'latitude': 43.576,
  'longitude': 3.965,
  'useCount': 0,
  'id': '34154001',
  'name': 'MONTPELLIER-AEROPORT',
  'quality': 100,
  'contribution': 0.0},
 '34274001': {'distance': 22177.0,
  'latitude': 43.78,
  'longitude': 3.729,
  'useCount': 0,
  'id': '34274001',
  'name': 'ST MARTIN DE LONDRES',
  'quality': 100,
  'contribution': 0.0},
 'LFMT': {'distance': 8314.0,
  'latitude': 43.58,
  'longitude': 3.97,
  'useCount': 0,
  'id': 'LFMT',
  'name': 'LFMT',
  'quality': 50,
  'contribution': 0.0},
 '34217001': {'distance': 11980.0,
  'latitude': 43.718,
  'longitude': 3.867,
  'useCount': 0,
  'id': '34217001',
  'name': 'PRADES LE LEZ',
  'quality': 100,
  'contribution': 0.0},
 '07646099999': {'distance': 46446.0,
  'latitude': 43.757,
  'longitude': 4.416,
  

In [9]:
mpl_df = pd.DataFrame(mpl_data['days'])
mpl_df.head()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
0,2025-06-15,1749938400,31.0,20.6,25.7,32.8,20.6,26.4,18.5,65.5,...,06:02:38,1749960158,21:27:45,1750015665,0.65,"Rain, Partially cloudy",Partly cloudy throughout the day with afternoo...,rain,"[30003001, 34154001, 34274001, 34217001, D7130...",obs
1,2025-06-16,1750024800,30.8,19.4,25.6,29.6,19.4,25.3,13.1,48.2,...,06:02:39,1750046559,21:28:08,1750102088,0.68,Clear,Clear conditions throughout the day.,clear-day,"[30003001, 34154001, 34274001, 34217001, D7130...",obs
2,2025-06-17,1750111200,30.5,18.3,25.0,29.8,18.3,24.9,13.2,48.9,...,06:02:43,1750132963,21:28:29,1750188509,0.72,Clear,Clear conditions throughout the day.,clear-day,"[30003001, 34154001, 34274001, 34217001, D7130...",obs
3,2025-06-18,1750197600,31.6,17.8,25.5,31.4,17.8,25.6,16.3,59.8,...,06:02:49,1750219369,21:28:48,1750274928,0.75,Clear,Clear conditions throughout the day.,clear-day,"[30003001, 34154001, 34274001, 34217001, D7130...",obs
4,2025-06-19,1750284000,30.8,17.9,25.6,31.3,17.9,25.9,16.4,58.7,...,06:02:57,1750305777,21:29:05,1750361345,0.79,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[30003001, 34154001, 34274001, 34217001, D7130...",obs


In [22]:
mpl_df.tail()

,datetime,datetimeEpoch,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,dew,humidity,...,sunrise,sunriseEpoch,sunset,sunsetEpoch,moonphase,conditions,description,icon,stations,source
42,2025-07-27,1753567200,28.2,21.5,25.2,27.6,21.5,25.2,13.1,47.7,...,06:28:38,1753590518,21:12:57,1753643577,0.08,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"[30003001, 34154001, 34274001, 34217001, D7130...",obs
43,2025-07-28,1753653600,28.1,20.1,24.3,27.1,20.1,24.1,9.3,39.2,...,06:29:40,1753676980,21:11:51,1753729911,0.12,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,"[30003001, 34154001, 34274001, LFMT, 34217001,...",obs
44,2025-07-29,1753740000,29.9,19.9,24.9,28.4,19.9,24.4,7.8,34.8,...,06:30:42,1753763442,21:10:45,1753816245,0.15,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"[LFMT, D7130, LFTW]",obs
45,2025-07-30,1753826400,29.4,20.9,25.1,28.2,20.9,24.7,10.9,41.7,...,06:31:45,1753849905,21:09:36,1753902576,0.18,Partially cloudy,Becoming cloudy in the afternoon.,partly-cloudy-day,"[LFMT, D7130, LFTW]",comb
46,2025-07-31,1753912800,30.0,20.6,25.3,28.9,20.6,25.1,13.6,49.7,...,06:32:49,1753936369,21:08:26,1753988906,0.21,Partially cloudy,Partly cloudy throughout the day.,partly-cloudy-day,None,fcst


In [11]:
mpl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 36 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   datetime        47 non-null     object 
 1   datetimeEpoch   47 non-null     int64  
 2   tempmax         47 non-null     float64
 3   tempmin         47 non-null     float64
 4   temp            47 non-null     float64
 5   feelslikemax    47 non-null     float64
 6   feelslikemin    47 non-null     float64
 7   feelslike       47 non-null     float64
 8   dew             47 non-null     float64
 9   humidity        47 non-null     float64
 10  precip          47 non-null     float64
 11  precipprob      47 non-null     float64
 12  precipcover     47 non-null     float64
 13  preciptype      17 non-null     object 
 14  snow            47 non-null     float64
 15  snowdepth       47 non-null     float64
 16  windgust        47 non-null     float64
 17  windspeed       47 non-null     float

In [10]:
mpl_df.loc[0, 'stations']

['30003001',
 '34154001',
 '34274001',
 '34217001',
 'D7130',
 '07643099999',
 '30352002']

In [24]:
stations_df = mpl_df.loc[0:45,'stations'].map(lambda x: len(x))
stations_df

,stations
0,"[30003001, 34154001, 34274001, 34217001, D7130..."
1,"[30003001, 34154001, 34274001, 34217001, D7130..."
2,"[30003001, 34154001, 34274001, 34217001, D7130..."
3,"[30003001, 34154001, 34274001, 34217001, D7130..."
4,"[30003001, 34154001, 34274001, 34217001, D7130..."
5,"[30003001, 34154001, 34274001, 34217001, D7130..."
6,"[30003001, 34154001, 34274001, 34217001, D7130..."
7,"[30003001, 34154001, 34274001, 34217001, D7130..."
8,"[30003001, 34154001, 34274001, 34217001, D7130..."
9,"[30003001, 34154001, 34274001, 34217001, D7130..."


In [25]:
mpl_df.loc[0:45,'stations'].map(lambda x: len(x))

,stations
0,7
1,8
2,7
3,7
4,7
5,7
6,8
7,7
8,8
9,7
